## Result 6: neuron clusters
Investigate the expression of MOD markers in neuronal clusters identified by Moffitt et al. using scRNA-seq data.

In [ ]:
from pathlib import Path

import sys
import os
sys.path.append(os.path.abspath("../src"))

from utils import load_scRNA_data
from utils import order_neuron_clusters, get_cluster_boundaries
from plot import plot_neuron_cluster_heatmap, CM

import numpy as np
import pandas as pd

### data

#### Matched Single-Cell RNA Sequencing

In [ ]:
# Define file paths
single_cell_data_folder_path = Path("../data/mouse_hypothalamus/SingleCell/")
mtx_path = single_cell_data_folder_path / "GSE113576_matrix.mtx"
barcodes_path = single_cell_data_folder_path / "GSE113576_barcodes.tsv"
genes_path = single_cell_data_folder_path / "GSE113576_genes.tsv"
meta_path = single_cell_data_folder_path / "aau5324_Moffitt_Table-S1.xlsx"

In [ ]:
cell_class_dict = {
    'Excitatory': 'Excitatory',
    'Inhibitory': 'Inhibitory',
}

adata = load_scRNA_data(
    mtx_path, 
    barcodes_path, 
    genes_path, 
    meta_path, 
    cell_class_filter = cell_class_dict,
    neuron_cluster=True
)

In [ ]:
sc_data = adata.to_df()
neuron_cluster = adata.obs["Neuronal_cluster"]
sc_data["Neuronal_cluster"] = neuron_cluster

In [ ]:
from natsort import natsorted

clusters = sc_data['Neuronal_cluster'].unique()
sorted_clusters = natsorted(clusters)

sorted_index = []
for cluster in sorted_clusters:
    cells = sc_data.index[sc_data['Neuronal_cluster'] == cluster]
    sorted_index.extend(cells)

sc_data_sorted = sc_data.loc[sorted_index]

#### Marker Genes

differentially expressed genes identified by BANKSY

In [ ]:
# all differentially expressed genes
DE_genes = ['Mbp', 'Lpar1', 'Trh', 'Ucn3', 'Cck', 'Mlc1', 'Dgkk', 'Cbln2', 'Syt4', 'Gad1', 'Plin3', 'Gnrh1', 'Sln', 'Gjc3']
# DE_genes_MOD2: 7
DE_genes_MOD2 = ['Mlc1', 'Dgkk', 'Cbln2', 'Syt4', 'Gad1', 'Plin3', 'Gnrh1', 'Sln', 'Gjc3']
# DE_genes_MOD1: 8
DE_genes_MOD1 = ['Mbp', 'Lpar1', 'Trh', 'Ucn3', 'Cck']

In [ ]:
Gene_Group = pd.DataFrame([0,0,0,0,0,1,1,1,1,1,1,1,1,1], columns=['cluster'])

In [ ]:
# Excitatory
Ex_sc_data = sc_data_sorted[sc_data_sorted['Neuronal_cluster'].str.startswith("e")]
sc_MOD2 = Ex_sc_data.loc[:, [gene for gene in DE_genes_MOD2 if gene in Ex_sc_data.columns]]
sc_MOD1 = Ex_sc_data.loc[:, [gene for gene in DE_genes_MOD1 if gene in Ex_sc_data.columns]]
Ex_DE = pd.concat([sc_MOD1, sc_MOD2], axis=1)

Ex_cluster = Ex_sc_data["Neuronal_cluster"]

In [ ]:
Ex_DE.shape

In [ ]:
# Inhibitory
In_sc_data = sc_data_sorted[sc_data_sorted['Neuronal_cluster'].str.startswith("i")]
sc_MOD2 = In_sc_data.loc[:, [gene for gene in DE_genes_MOD2 if gene in In_sc_data.columns]]
sc_MOD1 = In_sc_data.loc[:, [gene for gene in DE_genes_MOD1 if gene in In_sc_data.columns]]
In_DE = pd.concat([sc_MOD2, sc_MOD1], axis=1)

In_cluster = In_sc_data["Neuronal_cluster"]

In [ ]:
In_DE.shape

In [ ]:
# Hybrid
Hy_sc_data = sc_data_sorted[sc_data_sorted['Neuronal_cluster'].str.startswith("h")]
sc_MOD2 = Hy_sc_data.loc[:, [gene for gene in DE_genes_MOD2 if gene in Hy_sc_data.columns]]
sc_MOD1 = Hy_sc_data.loc[:, [gene for gene in DE_genes_MOD1 if gene in Hy_sc_data.columns]]
Hy_DE = pd.concat([sc_MOD2, sc_MOD1], axis=1)

Hy_cluster = Hy_sc_data["Neuronal_cluster"]


In [ ]:
Hy_DE.shape

In [ ]:
del sc_data, sc_data_sorted, sorted_clusters, sorted_index, sc_MOD1, sc_MOD2, adata

### Excitatory

In [ ]:
re_Ex_DE, re_Ex_clu = order_neuron_clusters(cluster_data=Ex_DE, cluster_labels=Ex_cluster, gene_groups=Gene_Group, zscore=True)

In [ ]:
cluster_info = get_cluster_boundaries(re_Ex_clu)
print(cluster_info)

In [ ]:
plot_neuron_cluster_heatmap(re_Ex_DE.iloc[:,:1169], re_Ex_clu[:1169], DE_g=True, figures=(18*CM,13*CM))

In [ ]:
plot_neuron_cluster_heatmap(re_Ex_DE.iloc[:,1169:3194], re_Ex_clu[1169:3194], DE_g=True, figures=(18*CM,23*CM))

In [ ]:
plot_neuron_cluster_heatmap(re_Ex_DE.iloc[:,3194:], re_Ex_clu[3194:], DE_g=True, figures=(18*CM,2.5*CM))

### Inhibitory

In [ ]:
re_IN_DE, re_IN_clu = order_neuron_clusters(cluster_data=In_DE, cluster_labels=In_cluster, gene_groups=Gene_Group, zscore=True)

In [ ]:
cluster_info = get_cluster_boundaries(re_IN_clu)
print(cluster_info)

In [ ]:
plot_neuron_cluster_heatmap(re_IN_DE.iloc[:,:2269], re_IN_clu[:2269], DE_g=True, figures=(18*CM,9*CM))

In [ ]:
plot_neuron_cluster_heatmap(re_IN_DE.iloc[:,2269:8148], re_IN_clu[2269:8148], DE_g=True, figures=(18*CM,23*CM))

In [ ]:
plot_neuron_cluster_heatmap(re_IN_DE.iloc[:,8148:], re_IN_clu[8148:], DE_g=True, figures=(18*CM,22*CM))

### Hybrid

In [ ]:
re_Hy_DE, re_Hy_clu = order_neuron_clusters(cluster_data=Hy_DE, cluster_labels=Hy_cluster, gene_groups=Gene_Group, zscore=True)

In [ ]:
plot_neuron_cluster_heatmap(re_Hy_DE, re_Hy_clu, DE_g=True,  figures=(18*CM,3*CM))